In [3]:
import json
import os
from datasets import Dataset
from huggingface_hub import HfApi, create_repo, login
from typing import Dict, List
from dotenv import load_dotenv

class HuggingFaceDatasetCreator:
    def __init__(self):
        """
        Khởi tạo với token từ biến môi trường hoặc cache
        """
        # Load biến môi trường
        load_dotenv(override=True)
        
        # Thử lấy token từ biến môi trường
        self.token = os.getenv('HF_TOKEN')
        
        if not self.token:
            raise ValueError("Không tìm thấy token. Hãy chạy 'huggingface-cli login' hoặc thêm HF_TOKEN vào file .env")
            
        # Login vào Hugging Face
        try:
            self.api = HfApi(token=self.token)
            # Kiểm tra token có hợp lệ không
            user_info = self.api.whoami()
            print(f"✅ Đăng nhập thành công với tài khoản: {user_info['name']}")
        except Exception as e:
            raise ValueError(f"Lỗi đăng nhập: {str(e)}")
    
    def read_json_dataset(self, file_path: str) -> Dict[str, List]:
        """
        Đọc file JSON dataset theo format Alpaca
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                
            formatted_data = {
                'instruction': [],
                'input': [],
                'output': []
            }
            
            for item in data:
                formatted_data['instruction'].append(item['instruction'])
                formatted_data['input'].append(item['input'])
                formatted_data['output'].append(item['output'])
                
            return formatted_data
        except Exception as e:
            raise ValueError(f"Lỗi đọc file JSON: {str(e)}")
    
    def create_and_push_dataset(self, data: Dict[str, List], json_file: str, description: str = None):
        """
        Tạo và đẩy dataset lên Hugging Face Hub
        """
        try:
            # Tạo dataset
            dataset = Dataset.from_dict(data)
            
            # Lấy thông tin repository từ biến môi trường
            username = os.getenv('HF_USERNAME')
            
            # Lấy tên dataset từ tên file JSON (không có phần mở rộng)
            dataset_name = os.path.splitext(os.path.basename(json_file))[0]
            
            if not username or not dataset_name:
                raise ValueError("Thiếu HF_USERNAME trong file .env")
                
            repo_name = f"{username}/{dataset_name}"
            
            # Tạo repository
            try:
                create_repo(
                    repo_id=repo_name,
                    repo_type="dataset",
                    private=True,
                    token=self.token,
                    description=description
                )
                print(f"✅ Đã tạo repository: {repo_name}")
            except Exception as e:
                print(f"⚠️ Repository có thể đã tồn tại: {str(e)}")
            
            # Đẩy dataset lên hub
            dataset.push_to_hub(repo_name, token=self.token)
            
            print(f"✅ Dataset đã được đăng tải thành công tại: https://huggingface.co/datasets/{repo_name}")
            
        except Exception as e:
            raise ValueError(f"Lỗi khi tạo và đẩy dataset: {str(e)}")

def main():
    # Cấu hình
    JSON_FILE = "output_samples.json"
    DESCRIPTION = """
    Dataset văn phong Thích Nhất Hạnh được tạo từ các tác phẩm của tác giả.
    Format: Alpaca
    """
    
    try:
        # Khởi tạo creator
        creator = HuggingFaceDatasetCreator()
        
        # Đọc và xử lý dataset
        print("📚 Đang đọc file JSON...")
        data = creator.read_json_dataset(JSON_FILE)
        
        # Tạo và đẩy dataset
        print("⬆️ Đang đăng tải dataset lên Hugging Face...")
        creator.create_and_push_dataset(data, JSON_FILE, DESCRIPTION)  # Truyền JSON_FILE vào đây
        
    except Exception as e:
        print(f"❌ Lỗi: {str(e)}")

if __name__ == "__main__":
    main()

✅ Đăng nhập thành công với tài khoản: nphuc
📚 Đang đọc file JSON...
⬆️ Đang đăng tải dataset lên Hugging Face...
⚠️ Repository có thể đã tồn tại: HfApi.create_repo() got an unexpected keyword argument 'description'


Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.81s/it]


✅ Dataset đã được đăng tải thành công tại: https://huggingface.co/datasets/nphuc/output_samples
